In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.experiment import Experiment

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from deeprte.config import make_config
config = make_config()

In [4]:
e = Experiment(mode = "train", init_rng=0, config = config)

In [5]:
ds = e._build_train_input()

In [6]:
import tensorflow as tf
tf.nest.map_structure(lambda x: x.shape, next(ds))

{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'boundary': (1, 6, 1968),
 'scattering_kernel': (1, 6, 500, 24),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [7]:
ds = e._build_val_input()

In [8]:
tf.nest.map_structure(lambda x: x.shape, next(ds))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'boundary': (1, 2, 1968),
 'scattering_kernel': (1, 2, 40344, 24),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}